In [1]:
import time

import numpy as np

import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, Softmax
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.losses import Huber
from tensorflow.keras.metrics import AUC, BinaryAccuracy, Accuracy, Recall, Precision
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard, LearningRateScheduler, ReduceLROnPlateau

from HW_base import evaluate_build, focus_build
from HW_keras import hw_layer

In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [3]:
y_train = (y_train==0).astype(np.float32)
y_test = (y_test==0).astype(np.float32)

In [4]:
# y_train = np.concatenate((y_train==0, y_train != 0), axis=-1).astype(np.float32)
# y_test = np.concatenate((y_test==0, y_test != 0), axis=-1).astype(np.float32)

In [5]:
x_min = np.min(x_train, axis=(1, 2), keepdims=True)
x_max = np.max(x_train, axis=(1, 2), keepdims=True)
x_train = (x_train - x_min) / (x_max - x_min)

In [6]:
x_min = np.min(x_test, axis=(1, 2), keepdims=True)
x_max = np.max(x_test, axis=(1, 2), keepdims=True)
x_test = (x_test - x_min) / (x_max - x_min)

In [7]:
# evaluate_list = [evaluate_build(x_test[..., i], 8) for i in range(x_test.shape[-1])]
# evaluate_focus_list = [focus_build(evaluate, 0.8) for evaluate in evaluate_list]

In [8]:
x = Input(shape=((32, 32, 3)))
y = x
# y = hw_layer(evaluate_focus_list)(x)

y = Conv2D(32, 3, 2, 'same')(y)
y = Conv2D(8, 1, 1, 'valid')(y)
y = Softmax(axis=-1)(y)

y = Conv2D(32, 3, 2, 'same')(y)
y = Conv2D(8, 1, 1, 'valid')(y)
y = Softmax(axis=-1)(y)

y = Conv2D(32, 3, 2, 'same')(y)
y = Conv2D(8, 1, 1, 'valid')(y)
y = Softmax(axis=-1)(y)

y = Flatten()(y)
y = Dense(16)(y)
y = Dense(1)(y)
y = Activation('sigmoid')(y)

model = Model(x, y)
model.summary()

model.compile(loss=Huber(), optimizer=Adam(learning_rate=0.001), metrics=[AUC(), Recall()])

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 16, 16, 32)        896       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 16, 16, 8)         264       
_________________________________________________________________
softmax (Softmax)            (None, 16, 16, 8)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 8, 8, 32)          2336      
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 8, 8, 8)           264       
_________________________________________________________________
softmax_1 (Softmax)          (None, 8, 8, 8)           0     

In [9]:
model_name = time.strftime('CIFAR10_KERAS_HWNET%Y%m%d%H%M%S')

callback_list = [
#     ReduceLROnPlateau(monitor='loss', factor=0.7,  patience=4, min_lr=1e-7, verbose=True),
    EarlyStopping(monitor='loss', patience=32),
    ModelCheckpoint('models/%s_best.h5'%model_name, monitor='loss', save_best_only=True, verbose=False),
    TensorBoard(log_dir='./Log/%s'%model_name)
]

model.fit(x_train, y_train, epochs=200, batch_size=1000, validation_data=(x_test, y_test), callbacks=callback_list)

Epoch 1/200
50/50 [==============================] - 4s 24ms/step - loss: 0.0569 - auc: 0.4922 - recall: 0.0000e+00 - val_loss: 0.0451 - val_auc: 0.5000 - val_recall: 0.0000e+00
Epoch 2/200
50/50 [==============================] - 1s 15ms/step - loss: 0.0450 - auc: 0.4989 - recall: 0.0000e+00 - val_loss: 0.0450 - val_auc: 0.5000 - val_recall: 0.0000e+00
Epoch 3/200
50/50 [==============================] - 1s 15ms/step - loss: 0.0450 - auc: 0.5030 - recall: 0.0000e+00 - val_loss: 0.0450 - val_auc: 0.4997 - val_recall: 0.0000e+00
Epoch 4/200
50/50 [==============================] - 1s 15ms/step - loss: 0.0450 - auc: 0.5117 - recall: 0.0000e+00 - val_loss: 0.0450 - val_auc: 0.5299 - val_recall: 0.0000e+00
Epoch 5/200
50/50 [==============================] - 1s 15ms/step - loss: 0.0449 - auc: 0.6109 - recall: 0.0000e+00 - val_loss: 0.0446 - val_auc: 0.7711 - val_recall: 0.0000e+00
Epoch 6/200
50/50 [==============================] - 1s 15ms/step - loss: 0.0433 - auc: 0.7589 - recall: 0.000

Epoch 49/200
50/50 [==============================] - 1s 15ms/step - loss: 0.0316 - auc: 0.8420 - recall: 0.3412 - val_loss: 0.0309 - val_auc: 0.8546 - val_recall: 0.3290
Epoch 50/200
50/50 [==============================] - 1s 15ms/step - loss: 0.0316 - auc: 0.8445 - recall: 0.3366 - val_loss: 0.0311 - val_auc: 0.8515 - val_recall: 0.3670
Epoch 51/200
50/50 [==============================] - 1s 15ms/step - loss: 0.0316 - auc: 0.8443 - recall: 0.3356 - val_loss: 0.0309 - val_auc: 0.8559 - val_recall: 0.3750
Epoch 52/200
50/50 [==============================] - 1s 15ms/step - loss: 0.0314 - auc: 0.8477 - recall: 0.3384 - val_loss: 0.0307 - val_auc: 0.8582 - val_recall: 0.3290
Epoch 53/200
50/50 [==============================] - 1s 15ms/step - loss: 0.0312 - auc: 0.8495 - recall: 0.3458 - val_loss: 0.0305 - val_auc: 0.8618 - val_recall: 0.3640
Epoch 54/200
50/50 [==============================] - 1s 15ms/step - loss: 0.0312 - auc: 0.8514 - recall: 0.3462 - val_loss: 0.0307 - val_auc: 0.

Epoch 97/200
50/50 [==============================] - 1s 15ms/step - loss: 0.0272 - auc: 0.8952 - recall: 0.4592 - val_loss: 0.0277 - val_auc: 0.8964 - val_recall: 0.4700
Epoch 98/200
50/50 [==============================] - 1s 15ms/step - loss: 0.0272 - auc: 0.8948 - recall: 0.4606 - val_loss: 0.0279 - val_auc: 0.8957 - val_recall: 0.4080
Epoch 99/200
50/50 [==============================] - 1s 15ms/step - loss: 0.0271 - auc: 0.8959 - recall: 0.4598 - val_loss: 0.0281 - val_auc: 0.8921 - val_recall: 0.3910
Epoch 100/200
50/50 [==============================] - 1s 15ms/step - loss: 0.0271 - auc: 0.8963 - recall: 0.4598 - val_loss: 0.0278 - val_auc: 0.8939 - val_recall: 0.4390
Epoch 101/200
50/50 [==============================] - 1s 15ms/step - loss: 0.0270 - auc: 0.8971 - recall: 0.4654 - val_loss: 0.0277 - val_auc: 0.8957 - val_recall: 0.4270
Epoch 102/200
50/50 [==============================] - 1s 15ms/step - loss: 0.0269 - auc: 0.8972 - recall: 0.4628 - val_loss: 0.0276 - val_auc:

Epoch 145/200
50/50 [==============================] - 1s 15ms/step - loss: 0.0250 - auc: 0.9046 - recall: 0.5148 - val_loss: 0.0276 - val_auc: 0.8984 - val_recall: 0.4920
Epoch 146/200
50/50 [==============================] - 1s 15ms/step - loss: 0.0251 - auc: 0.9042 - recall: 0.5174 - val_loss: 0.0278 - val_auc: 0.8926 - val_recall: 0.4360
Epoch 147/200
50/50 [==============================] - 1s 15ms/step - loss: 0.0250 - auc: 0.9041 - recall: 0.5156 - val_loss: 0.0276 - val_auc: 0.8972 - val_recall: 0.4520
Epoch 148/200
50/50 [==============================] - 1s 15ms/step - loss: 0.0251 - auc: 0.9039 - recall: 0.5194 - val_loss: 0.0277 - val_auc: 0.8967 - val_recall: 0.4460
Epoch 149/200
50/50 [==============================] - 1s 15ms/step - loss: 0.0251 - auc: 0.9044 - recall: 0.5132 - val_loss: 0.0276 - val_auc: 0.8992 - val_recall: 0.4960
Epoch 150/200
50/50 [==============================] - 1s 15ms/step - loss: 0.0249 - auc: 0.9049 - recall: 0.5200 - val_loss: 0.0276 - val_a

KeyboardInterrupt: 